In [1]:
from enhance_ocod.labelling.ner_spans import lfs
from enhance_ocod.labelling.weak_labelling import (process_dataframe_batch, create_flat_tag, 
create_commercial_park_tag, remove_overlapping_spans, get_overlap_stats, remove_zero_length_spans, convert_weak_labels_to_standard_format, evaluate_weak_labels)
import json
import pandas as pd 


data_to_label = "data/training_data/ground_truth_dev_set_labels.csv"

#data_to_label = "data/enhanced_ocod_data_and_gt/ground_truth_dev_set_labels.csv"

gt_path =  "data/training_data/ner_ready/ground_truth_dev_set_labels.json"

In [2]:
df = pd.read_csv(data_to_label)
#df = pd.read_csv("data/OCOD_FULL_2022_02.csv").rename(columns = {'Property Address':'property_address'})
df = df['property_address'].drop_duplicates().reset_index(drop=True).to_frame()


df= create_flat_tag(df)

df = create_commercial_park_tag(df)

/teamspace/studios/this_studio/enhance_ocod/src/enhance_ocod/labelling/weak_labelling.py:273: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df['flat_tag'] = df[text_column].str.contains(
/teamspace/studios/this_studio/enhance_ocod/src/enhance_ocod/labelling/weak_labelling.py:281: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df['commercial_park_tag'] = df[text_column].str.contains(


In [3]:
processed_data = process_dataframe_batch(df, 
                           batch_size = 1000,
                           text_column = 'property_address',
                           include_function_name = False,
                           save_intermediate = False,
                           output_dir = None,
                           verbose = True)
                           
remove_zero_length_spans(processed_data)
remove_overlapping_spans(processed_data)

Processing batches: 100%|██████████| 2/2 [00:01<00:00,  1.36it/s]


Processing complete:
  - Processed 1998 rows successfully
  - Found 10995 total spans
  - Average 5.50 spans per row


In [4]:
get_overlap_stats(processed_data)

{'total_rows': 1998,
 'rows_with_overlaps': 0,
 'overlap_percentage': 0.0,
 'total_spans_before': 9478,
 'total_overlaps_removed': 0,
 'spans_after': 9478}

In [5]:
standardised_labels = convert_weak_labels_to_standard_format(processed_data)

overall_results, class_df = evaluate_weak_labels(processed_data, 
    gt_path, 
    "data/model_performance", 
    'regex_labels')

Evaluating noisy predictions on regex_labels set...
Loaded 2000 ground truth examples
Loaded 1998 predictions

REGEX_LABELS SET RESULTS:
Overall F1: 0.9178
Overall Precision: 0.9201
Overall Recall: 0.9156

Per-class results:
building_name   precision: 0.9025  recall: 0.8875  f1: 0.8950  support: 240
city            precision: 0.9792  recall: 0.9866  f1: 0.9829  support: 524
number_filter   precision: 0.9375  recall: 0.9091  f1: 0.9231  support: 33
postcode        precision: 1.0000  recall: 1.0000  f1: 1.0000  support: 209
street_name     precision: 0.9792  recall: 0.9449  f1: 0.9618  support: 599
street_number   precision: 0.7964  recall: 0.9776  f1: 0.8777  support: 268
unit_id         precision: 0.9706  recall: 0.4286  f1: 0.5946  support: 154
unit_type       precision: 0.3472  recall: 0.7143  f1: 0.4673  support: 35

Results saved:
  Overall: data/model_performance/regex_labels_overall_performance.csv
  Per-class: data/model_performance/regex_labels_class_performance.csv


In [26]:
import pandas as pd
from enhance_ocod.labelling.weak_labelling import apply_labelling_functions_to_row

# Your example string
text = 'flats 1-30 (inclusive) tanyard place, shifnal (tf11 8bd)'

# Create a pandas row (Series) as the function expects
row = pd.Series({'text': text})

# This is how you would call the function
result = apply_labelling_functions_to_row(row, include_function_name=False)


print("Input text:")
print(f"'{text}'")
print("\nLabeled output:")
result

Input text:
'flats 1-30 (inclusive) tanyard place, shifnal (tf11 8bd)'

Labeled output:


{'text': 'flats 1-30 (inclusive) tanyard place, shifnal (tf11 8bd)',
 'spans': [{'text': 'shifnal', 'start': 38, 'end': 45, 'label': 'city'},
  {'text': 'tf11 8bd', 'start': 47, 'end': 55, 'label': 'postcode'},
  {'text': 'tanyard place', 'start': 23, 'end': 36, 'label': 'street_name'},
  {'text': '1-30', 'start': 6, 'end': 10, 'label': 'street_number'},
  {'text': '1-30', 'start': 6, 'end': 10, 'label': 'unit_id'}]}

In [34]:
with open("data/training_data/ner_ready/ground_truth_dev_set_labels.json", "r") as file:
    gt_list = json.load(file)

In [33]:
# Run diagnostics
diagnosis = diagnose_class_performance(gt_list, standardised_labels, 'unit_id')


# Print examples
print_diagnostic_examples(diagnosis, max_examples=10)
# Or access the raw data
for item in diagnosis['incorrect_predictions'][:5]:
    print(f"GT: {item['ground_truth']}")
    print(f"Pred: {item['prediction']}")
    print(f"Indexes: GT={item['gt_idx']}, Pred={item['pred_idx']}")
    print()


=== DIAGNOSTIC SUMMARY FOR CLASS: UNIT_ID ===
Total examples with unit_id in ground truth: 331
Correct predictions: 277
Incorrect predictions: 54
Missing predictions: 0
False positives: 1
Accuracy: 0.8369

=== EXAMPLES FOR UNIT_ID ===

INCORRECT PREDICTIONS (showing up to 10):
--------------------------------------------------------------------------------
Example 1:
  Text: flats 1 to 10, the briar, 43 thicket road, sutton (sm1 4qy) and storage area
  Ground Truth unit_id: [{'text': '1 to 10', 'start': 6, 'end': 13, 'label': 'unit_id'}]
  Predicted unit_id: []
  GT Index: 9, Pred Index: 952

Example 2:
  Text: ground to fifth floor flats being 2 to 6 and 9 to 50 (inclusive) castle court, 1 brewhouse lane and 1 to 16 (inclusive), 2 brewhouse lane, london
  Ground Truth unit_id: [{'text': '2 to 6', 'start': 34, 'end': 40, 'label': 'unit_id'}, {'text': '9 to 50', 'start': 45, 'end': 52, 'label': 'unit_id'}, {'text': '1 to 16', 'start': 100, 'end': 107, 'label': 'unit_id'}]
  Predicted u

# Process the Feb 2022 dataset to create the weakly labelled training set

In [39]:
df = pd.read_csv('data/ocod_history/OCOD_FULL_2022_02.zip').rename(columns = {'Property Address':'property_address'})
#df = pd.read_csv("data/OCOD_FULL_2022_02.csv").rename(columns = {'Property Address':'property_address'})
df = df['property_address'].drop_duplicates().reset_index(drop=True).to_frame()


df= create_flat_tag(df)

df = create_commercial_park_tag(df)

processed_data = process_dataframe_batch(df, 
                           batch_size = 1000,
                           text_column = 'property_address',
                           include_function_name = False,
                           save_intermediate = False,
                           output_dir = None,
                           verbose = True)
                           
remove_zero_length_spans(processed_data)
remove_overlapping_spans(processed_data)

rows = []
for item in processed_data:

    for span in item["spans"]:
        rows.append({
            "datapoint_id": item['datapoint_id'],
            "start": span["start"],
            "end": span["end"],
            "text": span["text"],
            "label": span["label"],
            "property_address": item["text"],
        })

df2 = pd.DataFrame(rows)
df2.to_csv('data/training_data/weakly_labelled.csv')
df2

/tmp/ipykernel_452430/3814106699.py:1: DtypeWarning: Columns (24,28,30,32,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/ocod_history/OCOD_FULL_2022_02.zip').rename(columns = {'Property Address':'property_address'})
/teamspace/studios/this_studio/enhance_ocod/src/enhance_ocod/labelling/weak_labelling.py:273: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df['flat_tag'] = df[text_column].str.contains(
/teamspace/studios/this_studio/enhance_ocod/src/enhance_ocod/labelling/weak_labelling.py:281: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df['commercial_park_tag'] = df[text_column].str.contains(
Processing batches: 100%|██████████| 93/93 [01:07<00:00,  1.37it/s]



Processing complete:
  - Processed 92361 rows successfully
  - Found 469087 total spans
  - Average 5.08 spans per row


,datapoint_id,start,end,text,label,property_address
0,0,0,25,Westleigh Lodge Care Home,building_name,"Westleigh Lodge Care Home, Nel Pan Lane, Leigh..."
1,0,27,39,Nel Pan Lane,street_name,"Westleigh Lodge Care Home, Nel Pan Lane, Leigh..."
2,0,41,46,Leigh,city,"Westleigh Lodge Care Home, Nel Pan Lane, Leigh..."
3,0,48,55,WN7 5JT,postcode,"Westleigh Lodge Care Home, Nel Pan Lane, Leigh..."
4,1,0,4,Flat,unit_type,"Flat 1, 1a Canal Street, Manchester (M1 3HE)"
...,...,...,...,...,...,...
414464,92360,8,10,17,unit_id,"Storage 17, Discovery Dock Apartments East, 3 ..."
414465,92360,12,42,Discovery Dock Apartments East,building_name,"Storage 17, Discovery Dock Apartments East, 3 ..."
414466,92360,46,63,South Quay Square,street_name,"Storage 17, Discovery Dock Apartments East, 3 ..."
414467,92360,65,71,London,city,"Storage 17, Discovery Dock Apartments East, 3 ..."


,datapoint_id,start,end,text,label,property_address
0,0,0,25,Westleigh Lodge Care Home,building_name,"Westleigh Lodge Care Home, Nel Pan Lane, Leigh..."
1,0,27,39,Nel Pan Lane,street_name,"Westleigh Lodge Care Home, Nel Pan Lane, Leigh..."
2,0,41,46,Leigh,city,"Westleigh Lodge Care Home, Nel Pan Lane, Leigh..."
3,0,48,55,WN7 5JT,postcode,"Westleigh Lodge Care Home, Nel Pan Lane, Leigh..."
4,1,0,4,Flat,unit_type,"Flat 1, 1a Canal Street, Manchester (M1 3HE)"
...,...,...,...,...,...,...
388923,92360,0,7,Storage,unit_type,"Storage 17, Discovery Dock Apartments East, 3 ..."
388924,92360,12,42,Discovery Dock Apartments East,building_name,"Storage 17, Discovery Dock Apartments East, 3 ..."
388925,92360,46,63,South Quay Square,street_name,"Storage 17, Discovery Dock Apartments East, 3 ..."
388926,92360,65,71,London,city,"Storage 17, Discovery Dock Apartments East, 3 ..."
